한국복지패널 데이터에서 한국인의 삶 분석하기

필요한 패키지 설치



In [1]:


!pip install pyreadstat
!pip install koreanize-matplotlib



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


라이브러리 인포트

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: ignored

In [22]:
# 2020년에 발간된 복지패널데이터 -> https://www.koweps.re.kr:442/data/data/list.do
# 통계 분석 소프트웨어 spss 전용 파일 사용 .sav
raw_welfare = pd.read_spss('/content/Koweps_hpwc14_2019_beta2.sav')
raw_welfare

PyreadstatError: ignored

In [32]:
# 복사본 만들기
welfare = raw_welfare.copy()
welfare

데이터 확인하기

In [24]:
welfare.info()

In [25]:
welfare.shape # (행 row, 열 cloumns) 출력

In [26]:
welfare.describe() # 범위 정보 출력 + 요약 통계량 보여줌

데이터 전처리

In [33]:
# 1) 컬럼명 변경하기
# 변경할 컬럼의 이름을 dictionary 자료구조를 사용
col_names = {'h14_g3' : 'sex',
            'h14_g4' : 'birth',
            'h14_g10' : 'marriage_type',
            'h14_g11' : 'religion',
            'p1402_8aq1' : 'income',
            'h14_eco9' : 'code_job',
            'h14_reg7' : 'region'} # 딕셔너리 변수 만들기

# 컬럼명 변경
welfare = welfare.rename(columns =col_names)


In [34]:
welfare[['sex', 'birth', 'marriage_type', 'religion', 'income', 'code_job', 'region']]

1. 성별에 따른 월급차이 - 성별에 따라 월급이 다를까?

In [35]:
# 1-1) 성별 데이터 확인하기

print('성별 데이터 타입 확인 : ', welfare['sex'].dtypes)
print('성별 데이터 결측치 확인 : ', welfare['sex'].isna().sum() ) # 결측치의 개수를 찾음
print('성별 빈도 확인')
welfare['sex'].value_counts()

In [36]:
# 1-20) 성별 데이터 전처리 -> 코드로 되어 있는 값(1.0, 2.0)을 문자열(male, female)
# 1.0 -> male, 2.0 -> female

# np.where(조건, True, False)
welfare['sex'] = np.where(welfare['sex'] == 1, 'male', 'female') # 위치 찾는 메소드 만족하면 첫번째 아니면 두번째

print('성별 빈도 확인')
welfare['sex'].value_counts()

In [ ]:
# 그래프로 확인


In [37]:
# 1-3) 월급 데이터 확인하기
print('월급 데이터 타입확인 : ', welfare['income'].dtypes)
print('월급 데이터 결측지 확인 : ', welfare['income'].isna().sum())
print('월급 요약 통계량 학인')
welfare['income'].describe()

In [38]:
# 그래프로 확인 -> 분포도
sns.histplot(data = welfare, x = 'income')
plt.show()

In [39]:
# 1-4) 월급 데이터 전처리 -> 결측치(NaN) 삭제 작업
welfare.dropna(subset = ['income'], inplace= True)
print('월급 데이터 결측지 확인 : ', welfare['income'].isna().sum())

In [44]:
# 1-5) 성별 월급 평균표 만들기
# Datafrme.groupby(기준이 되는 컬럼명) -> 남성과, 여성 2개의 rows
# 각 열마다 다른함수를 매칭 : group.agg(만들컬럼명 = (집계함수...))
sex_income = welfare.groupby('sex', as_index = False).agg(mean_income = ('income','mean')) # "as_index = False"는 컬럼으로 출력함.
sex_income

In [45]:
#welfare[['sex','income']]
# 성별에 따른 월급 차이 분석 -> 남성의 평균 월급이 여성의 두배 가까이 차이나는 결과값 확인 가능
sns.barplot(data = sex_income, x = 'sex', y = 'mean_income')
plt.show

2. 나이와 월급의 관계 - 최고 월급의 나이의 평균은 언제일까?

In [46]:
# 2-1) 나이 데이터 확인하기
print('나이 데이터 타입확인 : ', welfare['birth'].dtypes)
print('나이 데이터 결측지 확인 : ', welfare['birth'].isna().sum())
print('나이 요약 통계량 학인')
welfare['birth'].describe()

In [52]:
# 그래프로 확인하기 -> 나이의 분포 확인
sns.histplot(data = welfare, x = 'birth')
plt.show

In [50]:
# 2-2) 생년월일 전처리 -> 파생 데이터 -> 생년월일을 활용하여 나이데이터 만들기
# (2019 - 생년월일 + 1)
#assign(컬럼명 = 값) -> 새로운 컬럼으로 추가한 dataframe 반환
welfare = welfare.assign(age = 2019 - welfare['birth'] + 1)
welfare['age']

In [54]:
# 그래프로 확인하기
sns.histplot(data = welfare, x = 'age')
plt.show()

In [56]:
# 2-3) 나이에 따른 월급 평균표 만들기
age_income = welfare.groupby('age').agg(mean_income = ('income', 'mean'))
age_income.head() # 데이터 확인

In [57]:
# 2-4) 나이와 월급의 관계 분석 -> 최고 월급의 나이
# 그래프로 확인하기
sns.lineplot(data = age_income, x = 'age', y = 'mean_income')
plt.show()

In [60]:
# 가장 월급을 많이 받는 나이 -> 44세 월급 확인 -> 44세 평균월급 -> 386.195876 확인
age_income.loc[age_income.idxmax(),['mean_income']]

3. 연령대에 따른 월급차이 - 어떤 연령대가 월급을 가장 많이 받을까?

In [61]:
# 3-1) 나이 데이터 확인
# 연령댕 초년층 : 30세 미만, 중년층 : 30~ 59세, 노년층: 60세 이상
welfare['age'].head()

In [63]:
# 3-2) 연령대 데이터 전처리 -> 나이에서 파생 데이터를만들기 -> young, middle, old

np.where(welfare['age'] < 30, 'young', np.where(welfare['age'] < 60, 'middle', 'old'))

welfare = welfare.assign(ageg = np.where(welfare['age'] < 30, 'young', 
                                         np.where(welfare['age'] < 60, 'middle', 'old')))


In [ ]:
welfare

In [65]:
welfare['ageg'].value_counts()

In [66]:
# 그래프로 확인하기
sns.countplot(data = welfare, x = 'ageg')
plt.show()

In [68]:
# 3-3) 연령대별 월급 평균표 만들기
ageg_income = welfare.groupby('ageg', as_index = False).agg(mean_income = ('income', 'mean'))
ageg_income

In [73]:
# 3-4) 연령대에 따른 월급 차이 분석하기 -> 가장 월급을 많이 받는 연령대는 중년층
sns.barplot(data = ageg_income, x = 'ageg', y = 'mean_income',
            order = ['young', 'middle', 'old'])
plt.show()

In [74]:
ageg_income['mean_income'].max()

In [75]:
# 가장 높은 평균월급과 차이를 볼 수 있는 컬럼
ageg_income['diff'] = ageg_income['mean_income'].sub(age_income['mean_income'].max()) # 다시 설명
ageg_income

4. 연령대 및 성별 월급 차이 - 성별 월급 차이는 연령별로 다를까?

In [77]:
# 4-1) 연령대 및 성별 월급 평균표
sex_income = welfare.groupby(['ageg', 'sex'], as_index = False).agg(mean_income = ('income', 'mean'))

In [80]:
# 4-2) 연령대 및  성별 월급 차이 분석 -> 모든 연령대에서 남성이 평균 급여가 높음
# 여려 열에서 집단으로 묶어서 세부 집단으로 시각화 hue
sns.barplot(data = sex_income, x = 'ageg', y = 'mean_income', hue = 'sex',
            order = ['young', 'middle', 'old'])
plt.show()

In [82]:
# 여려 열에서 집단으로 묶어서 보고 싶으면 x 와 hue 값을 바꿔라
sns.barplot(data = sex_income, x = 'sex', y = 'mean_income', hue = 'ageg')
plt.show()

In [83]:
# 4-3) 나이 및 성별 월급 평균표
sex_age_income = welfare.groupby(['age', 'sex'], as_index = False
                                 ).agg(mean_income = ('income', 'mean'))
sex_age_income

In [86]:
# 나이 및 성별 월급 차이 분석 -> 
#여겅과 남성 30대 초중반 월급의 차이가 없다가 점점 크게 벌어짐 -> 80대에 비슷해짐
sns.lineplot(data = sex_age_income, x = 'age', y = 'mean_income', hue = 'sex')
plt.show()

5. 직업별 월급 차이 - 어떤 직업이 월급을 가장 많이 받을까?

In [ ]:
# 5-1) 직업 데이터 확인하기
print('직업 데이터 타입 확인 : ', welfare['code_job'].dtypes)
print('직업 데이터 결측치 확인 : ', welfare['code_job'].isna().sum() ) # 결측치의 개수를 찾음
print('직업 빈도 확인')
welfare['code_job'].value_counts()

In [88]:
# 5-2) 직업 데이터 전처리 -> 코드로 되어 있는 부분을 실제 직업명 추가
list_job = pd.read_excel('/content/Koweps_Codebook_2019.xlsx',
                         sheet_name = '직종코드')
list_job.head()

In [89]:
# welfare 데이터 프레임에 code_job(기준)과 매칭되는 job 정보 합치기
welfare = welfare.merge(list_job, how = 'left', on = 'code_job')
welfare[['code_job', 'job']]

In [90]:
welfare[['code_job', 'job']].isna().sum() # NULL 처리

In [91]:
# 5-3) 직업별 월급 평균표 만들기
job_income = welfare.groupby('job', as_index = False).agg(mean_income = ('income', 'mean'))
job_income

In [92]:
# 월급 가장 많이 받는 직업
job_income.sort_values('mean_income', ascending = False)

In [93]:
# 5-4) 월급 많이 받는 직업 TOP 5
job_top10 = job_income.sort_values('mean_income', ascending = False).head(10)
job_top10

In [97]:
# 그래프로 확인하기
sns.barplot(data = job_top10, y = 'job', x = 'mean_income' )
plt.show()

In [96]:
# 5-4) 월급 가장 적게 받는 직업 TOP 10
job_button10 = job_income.sort_values('mean_income').head(10)
job_button10

In [98]:
# 그래프로 확인하기
sns.barplot(data = job_button10, y = 'job', x = 'mean_income' )
plt.show()

6. 성별 직업 빈도 - 성별로 어떤 직업이 가장 많을까?

In [103]:
# 6-1) 성별 직업 빈도표 -> 각 성별로 직업별 빈도를 구해서 상위 10개 추출
# 남성 데이터 중에서 직업 빈도수 계산 -> 상위 10개 추출
# 남성 직업 빈도표 <- 남성 데이터만 찾아서 직업으로 그룹핑 후 빈도수 계산 후 내림차순으로 정렬 10개 추출
job_male = welfare.query('sex == "male"').groupby('job', as_index = False).agg(n = ('job', 'count')).sort_values('n', ascending = False).head(10) # query = male 만 찾아줌
job_male

In [104]:
# 여성 데이터 중에서 직업 빈도수 계산 -> 상위 10개 추출

job_female = welfare.query('sex == "female"').groupby('job', as_index = False).agg(n = ('job', 'count')).sort_values('n', ascending = False).head(10) # query = male 만 찾아줌
job_female

In [106]:
# 6-2) 성별 직업 빈도 분석하기
# 남성 직업 빈도 분석하기 -> 경영 관련 사무원, 자동차 운전원이 많음
sns.barplot(data = job_male, x = 'n', y = 'job').set(title = '남성 직업 빈도 TOp 10')
plt.xlabel('빈도수')
plt.ylabel('직업')
plt.show()


In [110]:
plt.figure(figsize = (15,5))
sns.barplot(data = job_male, x = 'job', y = 'n').set(title = '남성 직업 빈도 TOp 10')
plt.xlabel('직업')
plt.ylabel('빈도수')
plt.show()

In [111]:
# 여성 직업 빈도 분석하기 -> 
# 그래프 표현
sns.barplot(data = job_female, x = 'n', y = 'job').set(title = '여성 직업 빈도 TOp 10')
plt.xlabel('빈도수')
plt.ylabel('직업')
plt.show()

In [116]:
# 여성 직업 빈도 분석하기 -> 
# 그래프 표현
plt.figure(figsize = (15,5))
sns.barplot(data = job_female, x = 'job', y = 'n').set(title = '여성 직업 빈도 TOp 10')
plt.xlabel('직업')
plt.ylabel('빈도수')
plt.show()

7. 종교 유무에 따른 이혼율 - 종교가 있으면 이혼을 덜 할까?

In [117]:
# 5-1) 종교 데이터 확인하기
print('종교 데이터 타입 확인 : ', welfare['religion'].dtypes)
print('종교 데이터 결측치 확인 : ', welfare['religion'].isna().sum() ) # 결측치의 개수를 찾음
print('종교 빈도 확인')
welfare['religion'].value_counts()

In [118]:
# 7-2) 종교 데이터 전처리 -> 1 = 있음 2 = 없음 -> 코드로 되어 있는 부분을 yes, no 문자열 형태로 변경
welfare['religion']  = np.where(welfare['religion'] == 1, 'yes', 'no')
print('종교 빈도 확인')
welfare['religion'].value_counts()

In [120]:
# 그래프로 확인하기
sns.countplot(data = welfare, x = re)
 plt.show()

In [121]:
# 파이차트로 비율
welfare['religion'].value_counts().plot.pie(autopct = '%.2f%%')
plt.show()

In [122]:
# 7-3) 혼인 상태 데이터 확인

print('혼인 상태 데이터 타입 확인 : ', welfare['marriage_type'].dtypes)
print('혼인 상태 데이터 결측치 확인 : ', welfare['marriage_type'].isna().sum() ) # 결측치의 개수를 찾음
print('혼인 상태 빈도 확인')
welfare['marriage_type'].value_counts()

In [124]:
# 7-4) 혼인 상태 데이터 전처리 -> 7가지 코드 중 1 = 배우자 있음 3 = 이혼 divorce, 그 외 코드는 etc
# 새로운 컬럼을 만들어서 담기

welfare['marriage'] = np.where(welfare['marriage_type'] == 1, 'marrige', np.where(welfare['marriage_type'] == 3, 'divorce', 'etc'))
welfare['marriage'].value_counts()

In [125]:
# 7-5) 혼인 상태 빈도표 만들기
n_divorce = welfare.groupby('marriage', as_index = False).agg(n = ('marriage', 'count'))
n_divorce

In [126]:
# 그래프로 확인하기
sns.barplot(data = n_divorce, x = 'marriage', y = 'n')
plt.show()

In [131]:
# 7-6) 종교 유무에 따른 이혼율표 만들기 -> 혼인 상태가 etc 경우를 제외하고 종교 유무 및 이혼 여부 비율 계산
rel_div = welfare.query('marriage != "etc"').groupby('religion', as_index = False)['marriage'].value_counts(normalize = True) # value_counts(normalize = True)면 비율 계산
rel_div

In [135]:
# 7-7) 이혼에 해당되는 값만 추출해서 백분율(소수점 첫째자리까지 반올림)
rel_div_pct = rel_div.query('marriage == "divorce"').assign(proportion = rel_div['proportion']*100).round(1) # round(n) n번쨰 반올림
rel_div_pct

In [137]:
# 7-8) 종교 유무에 따른 이혼율 분석하기 -> 종교가 있으면 이혼 율 7.7 
sns.barplot(data = rel_div_pct, x = 'religion', y = 'proportion')
plt.show()

In [140]:

rel_div

In [141]:
# 7-8) 결혼에 해당되는 값만 추출해서 백분율(소수점 첫째ㅉ자리까지 반올림)
mar_rel_div_pct = rel_div.query('marriage == "marrige"').assign(proportion = rel_div['proportion']*100).round(1) # round(n) n번쨰 반올림
mar_rel_div_pct

In [142]:
sns.barplot(data = mar_rel_div_pct, x = 'religion', y = 'proportion')
plt.show()